In [5]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
cap = cv2.VideoCapture(0)

while True:
    try:
        ret, frame = cap.read()
        cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    except Exception as e:
        print(f"Camera error: {str(e)}")
        cap.release()
        cap = cv2.VideoCapture(0)

cap.release()
cv2.destroyAllWindows()

# Angle Calculator

In [3]:
#define a function to calculate angles between joints
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

In [4]:
cap = cv2.VideoCapture(0)

# Curl counter variables
left_counter_SP = 0
left_stage_SP = None

right_counter_SP = 0
right_stage_SP = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for left hand
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Get coordinates for right hand
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angle for left hand
            left_angle_SP = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle_SP = calculate_angle(left_hip, left_shoulder, left_elbow)

            #Calculate angle for right hand
            right_angle_SP = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle_SP = calculate_angle(right_hip, right_shoulder, right_elbow)

            # Visualize angle
            #put angle on left elbow
            cv2.putText(image, str(round(left_angle_SP, 2)),
                           tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA
                                )

            if left_shoulder_angle_SP > 60:
                cv2.putText(image, str(round(left_shoulder_angle_SP, 2)),
                              tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA
                                    )
            # put angle on the right elbow
            cv2.putText(image, str(round(right_angle_SP,2)),
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA
                                )

            if right_shoulder_angle_SP > 60:
                cv2.putText(image, str(round(right_shoulder_angle_SP,2)),
                               tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA
                                   )

            if left_shoulder_angle_SP < 60:
                cv2.putText(image, str("Increase This Angle"),
                          tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA
                                )

            if right_shoulder_angle_SP < 60:
                cv2.putText(image, str("Increase This Angle"),
                          tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA
                                )

            if left_angle_SP > 70 and left_stage_SP == 'down' and left_shoulder_angle_SP > 60 and left_shoulder_angle_SP < 100:
                cv2.putText(image, str("Decrease This Angle"),
                          tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA
                                )

            if right_angle_SP > 70 and right_stage_SP == 'down' and right_shoulder_angle_SP > 60 and right_shoulder_angle_SP < 100:
                cv2.putText(image, str("Decrease This Angle"),
                          tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA
                                )


            # Curl counter logic for left hand
            if left_angle_SP > 165 and left_shoulder_angle_SP > 150:
                left_stage_SP = "up"
            if left_angle_SP < 70 and left_stage_SP =='up' and left_shoulder_angle_SP > 60:
                left_stage_SP="down"
                left_counter_SP += 1

            # Curl counter logic for right hand
            if right_angle_SP > 165 and right_shoulder_angle_SP > 150:
                right_stage_SP = "up"
            if right_angle_SP < 70 and right_stage_SP =='up' and right_shoulder_angle_SP > 60:
                right_stage_SP="down"
                right_counter_SP += 1

        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # Rep data for left
        cv2.putText(image, 'LEFT_REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.2, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(left_counter_SP),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, .5, (255,255,255), 2, cv2.LINE_AA)

        # Stage data for left
        cv2.putText(image, 'LEFT_STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, .2, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, left_stage_SP,
                    (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, .5, (255,255,255), 2, cv2.LINE_AA)

        # Rep data for right
        cv2.putText(image, 'RIGHT_REPS', (115,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.2, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(right_counter_SP),
                    (115,60),
                    cv2.FONT_HERSHEY_SIMPLEX, .5, (255,255,255), 2, cv2.LINE_AA)

        # Stage data for right
        cv2.putText(image, 'RIGHT_STAGE', (165,12),
                    cv2.FONT_HERSHEY_SIMPLEX, .2, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, right_stage_SP,
                    (165,60),
                    cv2.FONT_HERSHEY_SIMPLEX, .5, (255,255,255), 2, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

        cv2.imshow('Shoulder Press Checker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
          break

    cap.release()
    cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
